<a href="https://colab.research.google.com/github/efrainudg/Biblioteca-UDG-2025-ANTES-DE-INVENTARIO/blob/main/Comparador_de_Bibliograf%C3%ADa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import re
import unicodedata

def normalize_text(text):
    """
    Limpia el texto: minúsculas, elimina acentos y caracteres especiales.
    """
    if not isinstance(text, str):
        return ""
    # Normalizar unicode (quita acentos)
    text = unicodedata.normalize('NFKD', text).encode('ASCII', 'ignore').decode('utf-8')
    # Quitar caracteres no alfanuméricos (excepto espacios) y pasar a minúsculas
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text.lower())
    return text.strip()

def run_comparison():
    print("Cargando archivos...")

    # 1. Cargar Referencias
    try:
        # Leer references.csv como un archivo de texto plano para evitar errores de tokenización
        # si las líneas contienen comas o comillas que confunden al parser CSV.
        # Cada línea se considera una referencia completa.
        with open('references.csv', 'r', encoding='utf-8') as f:
            references_list = [line.strip() for line in f if line.strip()]
        df_refs = pd.DataFrame(references_list, columns=['Reference Content'])
        ref_col = 'Reference Content'

        print(f"Referencias cargadas: {len(df_refs)} registros.")
    except Exception as e:
        print(f"Error cargando references.csv: {e}")
        return

    # 2. Cargar Libros (Catalogo)
    try:
        # El nombre del archivo es largo, usamos el nombre exacto proporcionado
        libros_filename = 'libros.xlsx - 4mr2rx3a10lhxkh4dnm3i0p3.csv.csv'
        # Se añade encoding='latin1' para manejar caracteres especiales que causan errores de decodificación.
        df_libros = pd.read_csv(libros_filename, encoding='latin1')

        # Arreglar nombres de columnas si vienen con codificación extraña (ej. TÃ­tulo)
        df_libros.columns = [c.replace('Ã­', 'i').replace('Ã³', 'o') for c in df_libros.columns]

        # Identificar columna de Título y Autor
        title_col = [c for c in df_libros.columns if 'titulo' in c.lower()]
        title_col = title_col[0] if title_col else df_libros.columns[2] # Fallback a columna 2

        author_col = [c for c in df_libros.columns if 'autor' in c.lower()]
        author_col = author_col[0] if author_col else df_libros.columns[3]

        print(f"Catálogo de libros cargado: {len(df_libros)} registros.")
    except Exception as e:
        print(f"Error cargando el catálogo de libros: {e}")
        return

    print("\nProcesando y comparando datos (esto puede tomar un momento)...")

    # 3. Pre-procesamiento para optimizar velocidad
    # Filtramos libros sin título o nulos
    libros_validos = df_libros[df_libros[title_col].notna()].copy()
    libros_validos = libros_validos[libros_validos[title_col] != '[NULO]']

    # Creamos columnas normalizadas
    libros_validos['titulo_norm'] = libros_validos[title_col].apply(normalize_text)
    # Filtramos títulos muy cortos (menos de 4 letras) para evitar falsos positivos (ej. libro llamado "Yo")
    libros_validos = libros_validos[libros_validos['titulo_norm'].str.len() > 3]

    results = []

    # Contadores
    found_count = 0
    missing_count = 0

    # 4. Lógica de Comparación
    for idx, row in df_refs.iterrows():
        ref_original = row[ref_col]
        ref_norm = normalize_text(ref_original)

        match_status = "FALTANTE"
        match_detail = ""
        match_author = ""

        # Estrategia: Buscar si el TITULO del libro existe dentro de la REFERENCIA completa.
        # Esto funciona bien porque las referencias suelen ser: "Autor (Año). Título del libro. Editorial"

        # Filtramos posibles candidatos (contención de strings)
        # Nota: Iterar todo el catálogo por cada referencia es costoso (O(N*M)),
        # pero necesario para 'substring matching' sin librerías complejas de NLP.

        matches = []
        for _, libro in libros_validos.iterrows():
            if libro['titulo_norm'] in ref_norm:
                matches.append(libro)

        if matches:
            # Si hay múltiples coincidencias, tomamos la que tenga el título más largo
            # (para evitar que un libro llamado "Historia" coincida con "Historia de México")
            best_match = max(matches, key=lambda x: len(x['titulo_norm']))
            match_status = "ENCONTRADO"
            match_detail = best_match[title_col]
            match_author = best_match[author_col]
            found_count += 1
        else:
            missing_count += 1

        results.append({
            'Referencia Original': ref_original,
            'Estado': match_status,
            'Titulo en Biblioteca': match_detail,
            'Autor en Biblioteca': match_author
        })

    # 5. Guardar Resultados
    df_results = pd.DataFrame(results)
    output_filename = 'reporte_comparativo.csv'
    df_results.to_csv(output_filename, index=False, encoding='utf-8-sig')

    print("\n" + "="*30)
    print("RESUMEN DEL ANÁLISIS")
    print("="*30)
    print(f"Total Referencias Analizadas: {len(df_refs)}")
    print(f"✅ Encontradas en Biblioteca:  {found_count}")
    print(f"❌ No Encontradas (Faltantes): {missing_count}")
    print(f"Cobertura de la bibliografía: {round((found_count / len(df_refs)) * 100, 2)}%")
    print("="*30)
    print(f"\nArchivo generado exitosamente: {output_filename}")
    print("Descarga este archivo para ver el detalle fila por fila.")

    # Mostrar muestra de faltantes
    if missing_count > 0:
        print("\nEjemplos de referencias FALTANTES (Primeras 3):")
        for item in df_results[df_results['Estado'] == 'FALTANTE']['Referencia Original'].head(3):
            print(f" - {item[:80]}...")

if __name__ == "__main__":
    run_comparison()

Cargando archivos...
Referencias cargadas: 295 registros.


/tmp/ipython-input-3133291431.py:40: DtypeWarning: Columns (0,7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_libros = pd.read_csv(libros_filename, encoding='latin1')


Catálogo de libros cargado: 131682 registros.

Procesando y comparando datos (esto puede tomar un momento)...


KeyboardInterrupt: 